# Contextual Synonym Analysis (NLTK Version 2)

This notebook identifies interchangeable words using NLTK WordNet and edit distance (Version 2).

In [4]:
import json
from nltk.corpus import wordnet as wn
from collections import defaultdict

THRESHOLD = 1
def extract_vocab(json_path):
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except FileNotFoundError:
        return []
        
    vocab_keys = data.get('model', {}).get('vocab', {}).keys()
    
    # Fast set comprehension for filtering
    return {w for w in vocab_keys 
            if len(w) > 3 and w.isalpha() and not w.startswith("##")}

def find_contextual_synonyms(vocab_set):
    print(f"Building indices for {len(vocab_set)} words...")
    
    # Build Lookup Tables (The "Inverted Index")
    word_to_synsets = {}
    
    synset_to_words = defaultdict(list)
    
    for word in vocab_set:
        synsets = set(syn.name() for syn in wn.synsets(word))
        
        if synsets:
            word_to_synsets[word] = synsets
            for sid in synsets:
                synset_to_words[sid].append(word)
                
    print("Calculating overlaps...")
    
    results = []
    processed_pairs = set()

    # Iterate through words and find their "Contextual Twins"
    for word_a, contexts_a in word_to_synsets.items():
        
        candidates = set()
        for sid in contexts_a:
            candidates.update(synset_to_words[sid])

        
        for word_b in candidates:
            if word_a == word_b:
                continue
            # Sort pair to avoid duplicates: (A, B) and (B, A)
            pair_key = tuple(sorted((word_a, word_b)))
            if pair_key in processed_pairs:
                continue
            
            # Calculate Similarity (Jaccard Index)
            contexts_b = word_to_synsets[word_b]
            
            intersection = len(contexts_a.intersection(contexts_b))
            union = len(contexts_a)
            score = intersection / union
            
            if score >= THRESHOLD :
                # Adding pair key only if score > Threshold allows alg to look both ways if one word has less definitions
                processed_pairs.add(pair_key)
                results.append({
                    "word_a": word_a,
                    "word_b": word_b,
                    "score": round(score, 2),
                    "shared_contexts": intersection,
                    "total_contexts": union
                })

    return results


vocab = extract_vocab("./bert_tokenizer_uncased/tokenizer.json")
#vocab = None

if not vocab:
    vocab = {"looked", "expected", "movie"}

# Find Pairs
interchangeable_pairs = find_contextual_synonyms(vocab)
interchangeable_pairs.sort(key=lambda x: x['score'], reverse=True)

print(f"\nFound {len(interchangeable_pairs)} highly interchangeable pairs.\n")
print(f"{'Word A':<15} | {'Replacement':<15} | {'Score':<5} | {'Shared Meanings'}")
print("-" * 60)

for p in interchangeable_pairs[:4000]:
    print(f"{p['word_a']:<15} | {p['word_b']:<15} | {p['score']:<5} | {p['shared_contexts']}/{p['total_contexts']}")

Building indices for 17582 words...
Calculating overlaps...

Found 6386 highly interchangeable pairs.

Word A          | Replacement     | Score | Shared Meanings
------------------------------------------------------------
movements       | movement        | 1.0   | 11/11
slogan          | motto           | 1.0   | 1/1
centre          | centres         | 1.0   | 11/11
plot            | plots           | 1.0   | 8/8
stage           | stages          | 1.0   | 10/10
schemes         | scheme          | 1.0   | 7/7
animation       | animations      | 1.0   | 6/6
fellow          | fellows         | 1.0   | 7/7
graduates       | graduate        | 1.0   | 5/5
similarly       | likewise        | 1.0   | 1/1
seat            | seats           | 1.0   | 16/16
diamond         | diamonds        | 1.0   | 6/6
sticks          | stick           | 1.0   | 25/25
members         | member          | 1.0   | 5/5
raider          | raiders         | 1.0   | 2/2
petitioned      | petition        | 1.0   | 1/

In [14]:
def generate_replacement_map(pairs_list):
    queue = sorted(pairs_list, key=lambda x: x['score'], reverse=True)
    
    replacement_map = {}
    
    print(f"Starting reduction on {len(queue)} pairs...")
    
    while queue:
        # Pop the best remaining pair
        current_pair = queue.pop(0)
        
        loser = current_pair['word_a']
        winner = current_pair['word_b']
        
        # Add to Dictionary
        replacement_map[loser] = winner
        
        # Back-Propagate (Fix Chains)

        for key, val in replacement_map.items():
            if val == loser:
                replacement_map[key] = winner

        # Forward-Propagate (Update the waiting list)

        #I dont want to replace the remaing queue word 'next_word' a with the loser because loser is a subset of winner as well as next_word. but winner may not be a subset of next_word
        #If word_a == loser --> remove from the data set
        #However is next word is a subset of loser, that makes it a subset of winner
        new_queue = []
        for p in queue:
            # Swap 'loser' to 'winner' in all future pairs
            if p['word_a'] == loser:
                continue #do not add to queue
            if p['word_b'] == loser:
                p['word_b'] = winner
            
            # Only keep the pair if it's still useful
            if p['word_a'] != p['word_b']:
                new_queue.append(p)
                
        queue = new_queue
    return replacement_map


vocab_map = generate_replacement_map(interchangeable_pairs)


print(f"\nReduction Complete. Generated {len(vocab_map)} mappings.")
print(f"{'Original':<15} -> {'Replacement'}")
print("-" * 30)


for original, replacement in list(vocab_map.items())[:20]:
    print(f"{original:<15} -> {replacement}")

# Verify Chains are Flattened

chained_keys = [k for k, v in vocab_map.items() if v in vocab_map]
if len(chained_keys) == 0:
    print("\nSuccess: No daisy-chains found. All words map directly to their final target.")
else:
    print(f"\nWarning: {len(chained_keys)} chains remain.")

removed_words = list(vocab_map.keys())
mapping = vocab_map

Starting reduction on 6386 pairs...

Reduction Complete. Generated 3910 mappings.
Original        -> Replacement
------------------------------
movements       -> movement
slogan          -> motto
centre          -> centres
plot            -> plots
stage           -> stages
schemes         -> scheme
animation       -> animations
fellow          -> fellows
graduates       -> graduate
similarly       -> likewise
seat            -> seats
diamond         -> diamonds
sticks          -> stick
members         -> member
raider          -> raiders
petitioned      -> petition
retros          -> retro
wing            -> wings
apply           -> applied
complained      -> complaining

Success: No daisy-chains found. All words map directly to their final target.


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("./bert_tokenizer_uncased")
model_state = tokenizer.get_vocab()
for word in removed_words:
    model_state.pop(word, None)

print(model_state)
vocab_list = [token for token, idx in sorted(model_state.items(), key=lambda x: x[1])]
print(vocab_list)
print(len(vocab_list), "words in new vocab")

import json
with open("./wordnet_v2/filtered_tokenizer_vocab_wordnet_v2.json", "w", encoding="utf-8") as f:
    json.dump(vocab_list, f, ensure_ascii=False, indent=2)


with open("./wordnet_v2/removed_words_mapping_wordnet_v2.json", "w", encoding="utf-8") as f:
    json.dump(mapping, f, ensure_ascii=False, indent=2)

{'sins': 25500, '酔': 3734, '##hs': 14035, 'হ': 557, '##ं': 4475, 'subgen': 21216, 'verified': 29048, '運': 3694, '駙': 3955, '麾': 4025, 'გ': 841, 'twisted': 22094, 'randall': 20913, '婷': 2009, 'brill': 12293, 'medley': 17888, '騎': 3958, 'ridic': 25961, 'general': 6133, 'specific': 8626, 'ʾ': 223, '渣': 2769, 'worst': 10985, '##ham': 6545, 'フ': 1420, 'pab': 24277, 'doggett': 19251, '⋅': 1213, 'placement': 17686, '桓': 2561, 'polyg': 25563, 'graph': 9824, '##ಡ': 4749, 'sponsorship': 22420, '##su': 15388, 'patton': 17588, '##ʋ': 4698, '##rows': 13256, 'gaz': 13841, '##mal': 13477, 'lis': 9866, '##ending': 7156, 'bust': 17721, '橿': 2632, 'cutsc': 24447, '##9': 4215, 'terrible': 17797, 'jay': 10428, '屠': 2106, '軒': 3638, 'ey': 7217, '##ateg': 7055, '##iser': 7901, 'mysterio': 27102, '##inaw': 28281, 'kilometer': 28624, '迦': 3674, '愛': 2300, '##anted': 16049, '深': 2758, 'wald': 16625, 'rhode': 17729, 'easton': 28848, 'ᐊ': 976, '蘄': 3429, 'mans': 10616, '榊': 2605, '1978': 10456, 'patt': 13125, 'c